In [9]:
# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
 
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

movie_analysis_examples = [
    {
        "movie": "Inside Out 2",
        "answer": """
        Here is what I analysed:

        Director: Kelsey Mann
        Main Cast: Amy Poehler, Maya Hawke, Phyllis Smith, Lewis Black, Tony Hale
        Budget: $200 million
        Box Office: $1.699 billion
        Genre: Animated Coming-of-Age
        Synopsis: Riley navigates her teenage years with new emotions like Anxiety and Envy joining Joy, Sadness, and the original crew.

        """,
    },
    {
        "movie": "Soul",
        "answer": """
        Here is what I analysed:

        Director: Pete Docter
        Main Cast: Jamie Foxx, Tina Fey, Graham Norton, Rachel House, Alice Braga
        Budget: $150 million
        Box Office: $120 million
        Genre: Animated Fantasy Comedy-Drama
        Synopsis:  Joe Gardner, a passionate jazz musician, finds himself in the afterlife after an accident. Teaming up with soul 22, he embarks on a journey to discover the true meaning of life.
        
        """,
    }
]

movie_analysis_example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Analyse movie name {movie}"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=movie_analysis_example_prompt,
    examples=movie_analysis_examples,
)


final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie analysis. You explain about movie information precisely, and shortly."),
        example_prompt,
        ("human", "Analyse movie name {movie}"),
    ]
)

chain = final_prompt | chat

chain.invoke({"movie": "ode to my father"})





        Here is what I analysed:

        Director: Yoon Je-kyoon
        Main Cast: Hwang Jung-min, Kim Yunjin, Oh Dal-su, Jang Young-nam
        Budget: $16 million
        Box Office: $105 million
        Genre: Drama
        Synopsis: The film follows the life of a man named Deok-soo, who sacrifices everything to provide for his family during tumultuous times in South Korea, spanning from the Korean War to the modern era.

AIMessageChunk(content='\n        Here is what I analysed:\n\n        Director: Yoon Je-kyoon\n        Main Cast: Hwang Jung-min, Kim Yunjin, Oh Dal-su, Jang Young-nam\n        Budget: $16 million\n        Box Office: $105 million\n        Genre: Drama\n        Synopsis: The film follows the life of a man named Deok-soo, who sacrifices everything to provide for his family during tumultuous times in South Korea, spanning from the Korean War to the modern era.')